# Analysis of Brazilian Airports With Respect to Return to Land

## Library Import

Import the used libraries

In [1]:
import pandas as pd
import numpy
import os

import ipyleaflet
import geojson
import re

## Brazil Public Airport Database Pre-Processing 

Import the ANAC's public airport data

In [2]:
aeroportos = pd.read_csv('data/cadastro-de-aerodromos-civis-publicos.csv',
                         delimiter=";",
                         encoding = "cp1252",
                         header = 2,
                         usecols = ['CÓDIGO OACI','NOME','LATITUDE','LONGITUDE','ALTITUDE',
                                    'DESIGNAÇÃO', 'COMPRIMENTO', 'LARGURA',
                                    'DESIGNAÇÃO.1', 'COMPRIMENTO.1', 'LARGURA.1',
                                    'DESIGNAÇÃO.2', 'COMPRIMENTO.2', 'LARGURA.2'])
aeroportos

,CÓDIGO OACI,NOME,LATITUDE,LONGITUDE,ALTITUDE,DESIGNAÇÃO,COMPRIMENTO,LARGURA,DESIGNAÇÃO.1,COMPRIMENTO.1,LARGURA.1,DESIGNAÇÃO.2,COMPRIMENTO.2,LARGURA.2
0,SBAA,CONCEIÇÃO DO ARAGUAIA,8° 20' 55'' S,49° 18' 11'' W,199 m,08/26,1800 m,30 m,-,-,-,-,-,-
1,SBAE,BAURU/AREALVA,22° 9' 28'' S,49° 4' 6'' W,594 m,17/35,2010 m,45 m,-,-,-,-,-,-
2,SBAQ,BARTOLOMEU DE GUSMÃO,21° 48' 16'' S,48° 8' 25'' W,711 m,17/35,1800 m,30 m,-,-,-,-,-,-
3,SBAR,SANTA MARIA,10° 59' 7'' S,37° 4' 24'' W,7 m,11/29,2200 m,45 m,-,-,-,-,-,-
4,SBAT,PILOTO OSVALDO MARQUES DIAS,9° 51' 59'' S,56° 6' 18'' W,289 m,04/22,2500 m,30 m,-,-,-,-,-,-
5,SBAU,ESTADUAL DARIO GUARITA,21° 8' 39'' S,50° 25' 35'' W,415 m,05/23,2120 m,35 m,-,-,-,-,-,-
6,SBAX,ROMEU ZEMA,19° 33' 38'' S,46° 57' 56'' W,999 m,15/33,1900 m,30 m,-,-,-,-,-,-
7,SBBE,INTERNACIONAL DE BELÉM/VAL DE CANS/JÚLIO CEZAR...,1° 23' 5'' S,48° 28' 44'' W,17 m,06/24,2800 m,45 m,02/20,1830 m,45 m,-,-,-
8,SBBG,COMANDANTE GUSTAVO KRAEMER,31° 23' 27'' S,54° 6' 35'' W,186 m,06/24,1500 m,30 m,14/32,1149 m,45 m,-,-,-
9,SBBH,PAMPULHA - CARLOS DRUMMOND DE ANDRADE,19° 51' 7'' S,43° 57' 2'' W,789 m,13/31,2364 m,45 m,-,-,-,-,-,-


Converts altitude and runway length from meters (string) to feet (float)

In [3]:
def to_feet(x):
    if (x == '-'):
        return float('nan')
    else:
        return float(x.replace('m','').replace(',','.'))/0.3048

In [4]:
cols_in_m = ['ALTITUDE', 'COMPRIMENTO', 'COMPRIMENTO.1' , 'COMPRIMENTO.2']
for index in cols_in_m:
    aeroportos[index] = aeroportos[index].apply(to_feet)

Converts latitude and longitude data from degrees/minutes/seconds (string) to a decimal (float)

In [5]:
def to_dec(x):
    parts = re.split('[°\']+',x)
    if 'N' in parts[3]:
        return (int(parts[0]) + int(parts[1])/60.0 + float(parts[2])/3600.0)
    else:
        return -(int(parts[0]) + int(parts[1])/60.0 + float(parts[2])/3600.0)

In [6]:
cols_in_deg = ['LATITUDE', 'LONGITUDE']
for index in cols_in_deg:
    aeroportos[index] = aeroportos[index].apply(to_dec)
aeroportos

,CÓDIGO OACI,NOME,LATITUDE,LONGITUDE,ALTITUDE,DESIGNAÇÃO,COMPRIMENTO,LARGURA,DESIGNAÇÃO.1,COMPRIMENTO.1,LARGURA.1,DESIGNAÇÃO.2,COMPRIMENTO.2,LARGURA.2
0,SBAA,CONCEIÇÃO DO ARAGUAIA,-8.348611,-49.303056,652.887139,08/26,5905.511811,30 m,-,NaN,-,-,NaN,-
1,SBAE,BAURU/AREALVA,-22.157778,-49.068333,1948.818898,17/35,6594.488189,45 m,-,NaN,-,-,NaN,-
2,SBAQ,BARTOLOMEU DE GUSMÃO,-21.804444,-48.140278,2332.677165,17/35,5905.511811,30 m,-,NaN,-,-,NaN,-
3,SBAR,SANTA MARIA,-10.985278,-37.073333,22.965879,11/29,7217.847769,45 m,-,NaN,-,-,NaN,-
4,SBAT,PILOTO OSVALDO MARQUES DIAS,-9.866389,-56.105000,948.162730,04/22,8202.099738,30 m,-,NaN,-,-,NaN,-
5,SBAU,ESTADUAL DARIO GUARITA,-21.144167,-50.426389,1361.548556,05/23,6955.380577,35 m,-,NaN,-,-,NaN,-
6,SBAX,ROMEU ZEMA,-19.560556,-46.965556,3277.559055,15/33,6233.595801,30 m,-,NaN,-,-,NaN,-
7,SBBE,INTERNACIONAL DE BELÉM/VAL DE CANS/JÚLIO CEZAR...,-1.384722,-48.478889,55.774278,06/24,9186.351706,45 m,02/20,6003.937008,45 m,-,NaN,-
8,SBBG,COMANDANTE GUSTAVO KRAEMER,-31.390833,-54.109722,610.236220,06/24,4921.259843,30 m,14/32,3769.685039,45 m,-,NaN,-
9,SBBH,PAMPULHA - CARLOS DRUMMOND DE ANDRADE,-19.851944,-43.950556,2588.582677,13/31,7755.905512,45 m,-,NaN,-,-,NaN,-


Plot of all public airports. (Commented in this version)

In [7]:
# points = [tuple(x) for x in aeroportos[['LONGITUDE','LATITUDE']].values]
# dados = geojson.MultiPoint(points)
# m = ipyleaflet.Map(center = (-15.8267,-47.9218), zoom = 5)
# m.add_layer(ipyleaflet.GeoJSON(data=dados))
# m

For each airport, get the maximum runway length

In [8]:
aeroportos['COMPRIMENTO_MAX'] = aeroportos[['COMPRIMENTO', 'COMPRIMENTO.1', 'COMPRIMENTO.2']].max(axis=1)
aeroportos

,CÓDIGO OACI,NOME,LATITUDE,LONGITUDE,ALTITUDE,DESIGNAÇÃO,COMPRIMENTO,LARGURA,DESIGNAÇÃO.1,COMPRIMENTO.1,LARGURA.1,DESIGNAÇÃO.2,COMPRIMENTO.2,LARGURA.2,COMPRIMENTO_MAX
0,SBAA,CONCEIÇÃO DO ARAGUAIA,-8.348611,-49.303056,652.887139,08/26,5905.511811,30 m,-,NaN,-,-,NaN,-,5905.511811
1,SBAE,BAURU/AREALVA,-22.157778,-49.068333,1948.818898,17/35,6594.488189,45 m,-,NaN,-,-,NaN,-,6594.488189
2,SBAQ,BARTOLOMEU DE GUSMÃO,-21.804444,-48.140278,2332.677165,17/35,5905.511811,30 m,-,NaN,-,-,NaN,-,5905.511811
3,SBAR,SANTA MARIA,-10.985278,-37.073333,22.965879,11/29,7217.847769,45 m,-,NaN,-,-,NaN,-,7217.847769
4,SBAT,PILOTO OSVALDO MARQUES DIAS,-9.866389,-56.105000,948.162730,04/22,8202.099738,30 m,-,NaN,-,-,NaN,-,8202.099738
5,SBAU,ESTADUAL DARIO GUARITA,-21.144167,-50.426389,1361.548556,05/23,6955.380577,35 m,-,NaN,-,-,NaN,-,6955.380577
6,SBAX,ROMEU ZEMA,-19.560556,-46.965556,3277.559055,15/33,6233.595801,30 m,-,NaN,-,-,NaN,-,6233.595801
7,SBBE,INTERNACIONAL DE BELÉM/VAL DE CANS/JÚLIO CEZAR...,-1.384722,-48.478889,55.774278,06/24,9186.351706,45 m,02/20,6003.937008,45 m,-,NaN,-,9186.351706
8,SBBG,COMANDANTE GUSTAVO KRAEMER,-31.390833,-54.109722,610.236220,06/24,4921.259843,30 m,14/32,3769.685039,45 m,-,NaN,-,4921.259843
9,SBBH,PAMPULHA - CARLOS DRUMMOND DE ANDRADE,-19.851944,-43.950556,2588.582677,13/31,7755.905512,45 m,-,NaN,-,-,NaN,-,7755.905512


## Brazil's Regular Flights Database Preprocessing 

In [9]:
voos = pd.read_csv('data/diario.csv',
                    delimiter = ';',
                    usecols = ['Cód. Empresa','Nr. Voo','Equip.',
                               'Seg','Ter','Qua','Qui','Sex','Sáb','Dom','Cód Origem','Cód Destino'])
voos

,Cód. Empresa,Nr. Voo,Equip.,Seg,Ter,Qua,Qui,Sex,Sáb,Dom,Cód Origem,Cód Destino
0,AAF,0035,A332,0,0,0,0,0,6,0,LFPO,SBKP
1,AAF,0035,A332,0,2,0,4,5,0,7,LFPO,SBKP
2,AAF,0036,A332,0,0,0,0,0,6,0,SBKP,LFPO
3,AAF,0036,A332,0,2,0,4,5,0,7,SBKP,LFPO
4,AAL,0213,B752,1,2,3,4,5,6,7,KMIA,SBBR
5,AAL,0214,B752,1,2,3,4,5,6,7,SBBR,KMIA
6,AAL,0215,B789,0,2,0,4,0,6,7,KLAX,SBGR
7,AAL,0216,B789,0,2,0,4,0,6,7,SBGR,KLAX
8,AAL,0904,B772,1,2,3,4,5,6,7,SBGL,KMIA
9,AAL,0905,B772,1,2,3,4,5,6,7,KMIA,SBGL


### Get all equipment in operation in Brazil

The following code prints to 'data/equip.txt' all equipment codes available at the flighst database.

This was created so the type of each equipment was manually inserted.

This part of the code is commented for now. 

In [10]:
# equip = voos['Equip.'].unique()
# equip = numpy.sort(equip, axis = -1, kind='mergesort')
# equip.tofile('dados/equip.txt', sep='\n', format='%s')

## Aircraft type database importing

In [11]:
equip = pd.read_csv('data/equip.csv',
                        delimiter = ',')

## Databases Merge

Merge flights and equipment databases

In [12]:
database = pd.merge(voos,equip,how='inner',on=['Equip.'])
database

,Cód. Empresa,Nr. Voo,Equip.,Seg,Ter,Qua,Qui,Sex,Sáb,Dom,Cód Origem,Cód Destino,Type
0,AAF,0035,A332,0,0,0,0,0,6,0,LFPO,SBKP,Wide Body
1,AAF,0035,A332,0,2,0,4,5,0,7,LFPO,SBKP,Wide Body
2,AAF,0036,A332,0,0,0,0,0,6,0,SBKP,LFPO,Wide Body
3,AAF,0036,A332,0,2,0,4,5,0,7,SBKP,LFPO,Wide Body
4,AEA,0047,A332,0,0,0,0,5,6,0,LEMD,SBRF,Wide Body
5,AEA,0047,A332,1,0,0,0,0,0,0,LEMD,SBRF,Wide Body
6,AEA,0048,A332,0,2,0,0,0,6,7,SBRF,LEMD,Wide Body
7,AZA,0672,A332,1,0,3,0,5,6,0,LIRF,SBGL,Wide Body
8,AZA,0673,A332,0,2,0,4,0,6,7,SBGL,LIRF,Wide Body
9,AZU,8702,A332,0,2,0,4,5,0,0,SBKP,KFLL,Wide Body


Get the total of flights per aircraft type per airport and if there are daily flights

In [13]:
database = database.groupby(['Cód Origem','Type'],as_index=False)['Seg','Ter','Qua','Qui','Sex','Sáb','Dom'].sum()
database

,Cód Origem,Type,Seg,Ter,Qua,Qui,Sex,Sáb,Dom
0,CYYZ,Wide Body,1,2,3,4,5,6,7
1,DNMM,Wide Body,0,0,3,0,0,0,0
2,EBBR,Wide Body,0,0,0,4,0,6,0
3,EBLG,Wide Body,0,2,3,0,5,0,0
4,EDDF,Wide Body,4,8,15,12,20,30,28
5,EGKK,Wide Body,0,0,3,0,5,0,7
6,EGLL,Wide Body,3,6,9,12,15,18,21
7,EHAM,Wide Body,5,10,6,12,20,30,28
8,EKCH,Wide Body,0,0,0,4,0,0,0
9,ELLX,Wide Body,2,2,0,8,10,6,7


In [14]:
database['Voos Semanais'] = database['Seg'] + database['Ter']//2 + database['Qua']//3 + \
                       database['Qui']//4 + database['Sex']//5 + database['Sáb']//6 + database['Dom']//7
database['Diário'] = database[['Seg','Ter','Qua','Qui','Sex','Sáb','Dom']].gt(0).sum(axis=1) == 7
database

,Cód Origem,Type,Seg,Ter,Qua,Qui,Sex,Sáb,Dom,Voos Semanais,Diário
0,CYYZ,Wide Body,1,2,3,4,5,6,7,7,True
1,DNMM,Wide Body,0,0,3,0,0,0,0,1,False
2,EBBR,Wide Body,0,0,0,4,0,6,0,2,False
3,EBLG,Wide Body,0,2,3,0,5,0,0,3,False
4,EDDF,Wide Body,4,8,15,12,20,30,28,29,True
5,EGKK,Wide Body,0,0,3,0,5,0,7,3,False
6,EGLL,Wide Body,3,6,9,12,15,18,21,21,True
7,EHAM,Wide Body,5,10,6,12,20,30,28,28,True
8,EKCH,Wide Body,0,0,0,4,0,0,0,1,False
9,ELLX,Wide Body,2,2,0,8,10,6,7,9,False


Drop days of the week columns and merge with the airport database

In [15]:
database = database.drop(columns=['Seg','Ter','Qua','Qui','Sex','Sáb','Dom'])
database = pd.merge(database,aeroportos[['CÓDIGO OACI','COMPRIMENTO_MAX','LATITUDE','LONGITUDE']],
                    how='inner',
                    left_on=['Cód Origem'],right_on=['CÓDIGO OACI']).drop(columns=['CÓDIGO OACI'])
database

,Cód Origem,Type,Voos Semanais,Diário,COMPRIMENTO_MAX,LATITUDE,LONGITUDE
0,SBAC,Turboprop,2,False,5905.511811,-4.568611,-37.804722
1,SBAE,Narrow Body,2,False,6594.488189,-22.157778,-49.068333
2,SBAE,Turboprop,12,False,6594.488189,-22.157778,-49.068333
3,SBAR,Narrow Body,44,True,7217.847769,-10.985278,-37.073333
4,SBAR,Regional Jet,6,False,7217.847769,-10.985278,-37.073333
5,SBAR,Turboprop,23,False,7217.847769,-10.985278,-37.073333
6,SBAT,Regional Jet,7,True,8202.099738,-9.866389,-56.105000
7,SBAU,Turboprop,18,True,6955.380577,-21.144167,-50.426389
8,SBAX,Turboprop,1,False,6233.595801,-19.560556,-46.965556
9,SBBE,Narrow Body,151,True,9186.351706,-1.384722,-48.478889


Lista de Aeroportos com Voos Diários de Narrow Body com pista inferior a 6000ft

In [16]:
db_nb_6000 = database[(database['Type'] == 'Narrow Body') & 
            (database['COMPRIMENTO_MAX'] < 6000.0) &
            (database['Diário'] == True)]
db_nb_6000

,Cód Origem,Type,Voos Semanais,Diário,COMPRIMENTO_MAX,LATITUDE,LONGITUDE
20,SBCA,Narrow Body,14,True,5298.556430,-25.002222,-53.501944
44,SBCX,Narrow Body,17,True,5479.002625,-29.195556,-51.189722
85,SBIL,Narrow Body,19,True,5173.884514,-14.815000,-39.033333
99,SBJU,Narrow Body,29,True,5905.511811,-7.219167,-39.269444
101,SBJV,Narrow Body,32,True,5380.577428,-26.223056,-48.797778
104,SBKG,Narrow Body,7,True,5249.343832,-7.269167,-35.895000
126,SBMQ,Narrow Body,34,True,5905.511811,0.050833,-51.070278
130,SBNF,Narrow Body,88,True,5580.708661,-26.878611,-48.650833
155,SBQV,Narrow Body,14,True,5823.490814,-14.863611,-40.863056
165,SBRJ,Narrow Body,445,True,4340.551181,-22.910000,-43.162500
